In [115]:
# 1. 필요한 라이브러리 불러오기
import sklearn as sk
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import pickle

In [116]:
# 2. 데이터 불러오기
mh_prices = pd.read_csv('../../datasets/House Prices - Advanced Regression Techniques.csv')
mh_prices.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [117]:
mh_prices.info()  
# mh_prices.describe()
# mh_prices.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [118]:
df_mh_prices = pd.DataFrame(mh_prices)
df_mh_prices

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [119]:
# 고유값의 수로 데이터 타입 구분하기
def category_columns(df_mh_prices):
    continue_columns = []
    category_columns = []
    
    for column in df_mh_prices.columns :
        unique_values = df_mh_prices[column].nunique() # 고유값의 개수
        if unique_values < len(df_mh_prices) * 0.05: # 고유값이 데이터의 5% 미만이면 범주형으로 간주
            category_columns.append(column)
        else:
            continue_columns.append(column)
        
    return continue_columns, category_columns

continuous, categorical = category_columns(df_mh_prices)

# 일일이 확인하는건 비효율적임으로 통계를 활용한 고유값의 수로 데이터 타입 구분하기

In [120]:
# 연속형 범주형 확인
continuous

['Id',
 'LotFrontage',
 'LotArea',
 'YearBuilt',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GrLivArea',
 'GarageYrBlt',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 'ScreenPorch',
 'SalePrice']

In [121]:
mh_prices[[
            'Id', 'LotFrontage', 'LotArea', 'YearBuilt', 'MasVnrArea'
           ,'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF'
           , '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageYrBlt'
           , 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch'
           , 'ScreenPorch','SalePrice'
           ]].describe() 

,Id,LotFrontage,LotArea,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,70.049958,10516.828082,1971.267808,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,1515.463699,1978.506164,472.980137,94.244521,46.660274,21.954110,15.060959,180921.195890
std,421.610009,24.284752,9981.264932,30.202904,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,525.480383,24.689725,213.804841,125.338794,66.256028,61.119149,55.757415,79442.502883
min,1.000000,21.000000,1300.000000,1872.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,334.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34900.000000
25%,365.750000,59.000000,7553.500000,1954.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,1129.500000,1961.000000,334.500000,0.000000,0.000000,0.000000,0.000000,129975.000000
50%,730.500000,69.000000,9478.500000,1973.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,1464.000000,1980.000000,480.000000,0.000000,25.000000,0.000000,0.000000,163000.000000
75%,1095.250000,80.000000,11601.500000,2000.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,1776.750000,2002.000000,576.000000,168.000000,68.000000,0.000000,0.000000,214000.000000
max,1460.000000,313.000000,215245.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,5642.000000,2010.000000,1418.000000,857.000000,547.000000,552.000000,480.000000,755000.000000


In [122]:
# 결측치 확인 및 처리
mh_prices[[
            'Id', 'LotFrontage', 'LotArea', 'YearBuilt', 'MasVnrArea'
           ,'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF'
           , '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageYrBlt'
           , 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch'
           , 'ScreenPorch','SalePrice'
           ]].isnull().sum()

Id                 0
LotFrontage      259
LotArea            0
YearBuilt          0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
GrLivArea          0
GarageYrBlt       81
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
ScreenPorch        0
SalePrice          0
dtype: int64

In [123]:
# 결측치 처리 평균값으로 채우기
mh_prices['MasVnrArea'] = mh_prices['MasVnrArea'].fillna(mh_prices['MasVnrArea'].mean())
mh_prices['LotFrontage'] = mh_prices['LotFrontage'].fillna(mh_prices['LotFrontage'].mean())
mh_prices['GarageYrBlt'] = mh_prices['GarageYrBlt'].fillna(mh_prices['GarageYrBlt'].mean())

In [124]:
mh_prices.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

In [125]:
# 결측치 처리 후 데이터 확인
mh_prices[[
            'Id', 'LotFrontage', 'LotArea', 'YearBuilt', 'MasVnrArea'
           ,'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF'
           , '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageYrBlt'
           , 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch'
           , 'ScreenPorch','SalePrice'
           ]].head(3)

,Id,LotFrontage,LotArea,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,SalePrice
0,1,65.0,8450,2003,196.0,706,0,150,856,856,854,1710,2003.0,548,0,61,0,0,208500
1,2,80.0,9600,1976,0.0,978,0,284,1262,1262,0,1262,1976.0,460,298,0,0,0,181500
2,3,68.0,11250,2001,162.0,486,0,434,920,920,866,1786,2001.0,608,0,42,0,0,223500


In [126]:
# 1. 데이터 전처리 스케일링

# 연속형 데이터에 대해서만 스케일링
# scaler = StandardScaler()

# # 학습에 사용할 피쳐들만 선택하기 (연속형 데이터만)
# X = mh_prices[continuous].drop(columns=['SalePrice'])

# # 타겟 변수(예 : 'SalePrice')
# y = mh_prices['SalePrice']

# # 학습 데이터를 스케일링하기
# X_scaled = scaler.fit_transform(X)


In [127]:
# # 결측치 확인
# X_train.isnull().sum()
# X_test.isnull().sum()

In [128]:
# 훈련/테스트 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [129]:
# 결측치가 있는 열에 대해 평균값으로 채우기
# X_train = pd.DataFrame(X_train).fillna(X_train.mean())
# X_test = pd.DataFrame(X_test).fillna(X_test.mean())

In [130]:
# 연속형 데이터만 사용 
X = mh_prices[continuous] # 연속형 피쳐만 선택
# 목표값을 SalesPrice 사용
y = mh_prices['SalePrice'] # 목표값은 SalePrice

In [131]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   LotFrontage    1460 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   YearBuilt      1460 non-null   int64  
 4   MasVnrArea     1460 non-null   float64
 5   BsmtFinSF1     1460 non-null   int64  
 6   BsmtFinSF2     1460 non-null   int64  
 7   BsmtUnfSF      1460 non-null   int64  
 8   TotalBsmtSF    1460 non-null   int64  
 9   1stFlrSF       1460 non-null   int64  
 10  2ndFlrSF       1460 non-null   int64  
 11  GrLivArea      1460 non-null   int64  
 12  GarageYrBlt    1460 non-null   float64
 13  GarageArea     1460 non-null   int64  
 14  WoodDeckSF     1460 non-null   int64  
 15  OpenPorchSF    1460 non-null   int64  
 16  EnclosedPorch  1460 non-null   int64  
 17  ScreenPorch    1460 non-null   int64  
 18  SalePric

In [132]:
# 3. 모델 학습 및 예측

# SVR 모델 학습
svr = SVR()
svr.fit(X, y) # 전체 데이터에서 레이블이랑 피쳐만 구별해서 넣기
# 예측
y_pred_svr = svr.predict(X)
mean_squared_error(y, y_pred_svr)
r2_score(y, y_pred_svr)

-0.04719904663865915

In [133]:
# Linear Regression 모델 학습
lr = LinearRegression() # LinearRegression 모델 객체 생성
lr.fit(X,y) # 연속형 데이터를 사용하여 모델 학습

# 예측
y_pred_lr = lr.predict(X)

# 성능 평가
mean_squared_error(y, y_pred_lr)
r2_score(y, y_pred_lr)

1.0

In [134]:
# Random Forest 모델 학습
rf = RandomForestRegressor() # Random Forest 모델 객체 생성
rf.fit(X,y) # 연속형 데이터를 사용하여 모델 학습

# 예측
y_pred_rf = rf.predict(X)

# 성능 평가
mean_squared_error(y, y_pred_rf)
r2_score(y,y_pred_rf)


0.9995782348389676